In [1]:
import os
import torch

In [2]:
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    print("MPS device found.")
else:
    print("MPS device not found. Using CPU.")
    mps_device = torch.device("cpu")

# Example: Move a tensor to the MPS device
tensor = torch.ones(1, device=mps_device)

MPS device found.


In [5]:
# Install PyTorch if not already installed
#!pip install torch
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA device count: {torch.cuda.device_count()}")
    print(f"Current CUDA device: {torch.cuda.current_device()}")
    print(f"CUDA device name: {torch.cuda.get_device_name(0)}")
else:
    print("No GPU available, PyTorch will use CPU")

PyTorch version: 2.7.1
CUDA available: False
No GPU available, PyTorch will use CPU


In [7]:
from pathlib import Path

path = Path.cwd() / "data" / "Amazon.mat"
save_path = Path.cwd() / "result"
print(f"File {'exists' if os.path.exists(path) else 'does not exist'} at the specified path.")

print(f"File {'exists' if os.path.exists(save_path) else 'does not exist'} at the specified path.")

File exists at the specified path.
File exists at the specified path.


In [9]:

#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#print("Using device:", device)
device = torch.device("cpu")
print("Using device MPS is available using :", device)


Using device MPS is available using : cpu


In [11]:
import scipy.io
import os
import csv  # # updated here

file_path = path # '/content/drive/MyDrive/pes_sem4_basepaper1/data/Amazon.mat'

if os.path.exists(file_path):
  mat_data = scipy.io.loadmat(file_path)
  print("Keys in the .mat file:")
  print(mat_data.keys())
else:
  print(f"File not found at: {file_path}")

Keys in the .mat file:
dict_keys(['__header__', '__version__', '__globals__', 'Network', 'Label', 'Attributes'])


In [15]:
import numpy as np
import scipy.sparse as sp
import torch
import scipy.io as sio
import copy
import random
import os
import pickle as pkl


def load_dataset(args):
    config = dict()
    config['beta'] = 0
    config['norm'] = True
    config['clusters'] = args.clusters
    if args.dataset == 'Amazon':
        adj1, adj2, features, ano_label = load_mat(args.dataset)
        features, _ = preprocess_features(features)
        raw_features = features
        #config['cutting'] = 25
        config['cutting'] = min(getattr(args, 'cutting', 25), 25)  # updated here
        config['lamb'] = 1
        config['alpha'] = 0.8
        config['norm'] = False
    elif args.dataset == 'YelpChi':
        adj1, adj2, features, ano_label = load_mat(args.dataset)
        features, _ = preprocess_features(features)
        raw_features = features
        config['cutting'] = 3
        config['lamb'] = 1
        config['alpha'] = 0.8
        config['norm'] = False
    elif args.dataset == 'dblp':
        adj1, adj2, features, ano_label = load_dblp_graph()
        raw_features = features
        config['cutting'] = 20
        config['lamb'] = 0.01
        config['alpha'] = 1
    elif args.dataset == 'imdb':
        adj1, adj2, features, ano_label = load_imdb_graph()
        raw_features = features
        config['cutting'] = 15
        config['lamb'] = 0.01
        config['alpha'] = 1
    elif args.dataset == 'cert':
        adj1, adj2, features, ano_label = load_cert_graph()
        raw_features = features
        config['cutting'] = 7
        config['lamb'] = 0.1
        config['alpha'] = 0.8
    else:
        adj1, adj2, features, ano_label = load_mat(args.dataset)
        raw_features = features.todense()
        features = raw_features
        config['cutting'] = 7
        config['beta'] = 1
        if args.dataset == 'BlogCatalog':
            config['clusters'] = 5
            config['lamb'] = 0.01
            config['alpha'] = 0
    config['ft_size'] = features.shape[1]
    raw_adj1 = (adj1 + sp.eye(adj1.shape[0])).todense()
    raw_adj1 = torch.FloatTensor(raw_adj1[np.newaxis])
    if adj2 is None:
        raw_adj2 = None
    else:
        raw_adj2 = (adj2 + sp.eye(adj2.shape[0])).todense()
        raw_adj2 = torch.FloatTensor(raw_adj2[np.newaxis])
    raw_features = torch.FloatTensor(raw_features[np.newaxis])
    features = torch.FloatTensor(features[np.newaxis])
    return raw_features, features, adj1, adj2, ano_label, raw_adj1, raw_adj2, config

def sparse_to_tuple(sparse_mx, insert_batch=False):
    """Convert sparse matrix to tuple representation."""
    """Set insert_batch=True if you want to insert a batch dimension."""

    def to_tuple(mx):
        if not sp.isspmatrix_coo(mx):
            mx = mx.tocoo()
        if insert_batch:
            coords = np.vstack((np.zeros(mx.row.shape[0]), mx.row, mx.col)).transpose()
            values = mx.data
            shape = (1,) + mx.shape
        else:
            coords = np.vstack((mx.row, mx.col)).transpose()
            values = mx.data
            shape = mx.shape
        return coords, values, shape

    if isinstance(sparse_mx, list):
        for i in range(len(sparse_mx)):
            sparse_mx[i] = to_tuple(sparse_mx[i])
    else:
        sparse_mx = to_tuple(sparse_mx)

    return sparse_mx


def RandomDropEdge(x, adj_t, drop_percent=0.05):
    percent = drop_percent / 2
    row_idx, col_idx = x.nonzero().T

    index_list = []
    for i in range(len(row_idx)):
        index_list.append((row_idx[i], col_idx[i]))

    edge_num = int(len(row_idx) / 2)  # 9228 / 2
    add_drop_num = int(edge_num * percent / 2)
    aug_adj = copy.deepcopy(adj_t)

    edge_idx = [i for i in range(edge_num)]
    drop_idx = random.sample(edge_idx, add_drop_num)

    for i in drop_idx:
        aug_adj[index_list[i][0]][index_list[i][1]] = 0
        aug_adj[index_list[i][1]][index_list[i][0]] = 0

    node_num = x.shape[0]
    l = [(i, j) for i in range(node_num) for j in range(i)]
    add_list = random.sample(l, add_drop_num)

    for i in add_list:
        aug_adj[i[0]][i[1]] = 1
        aug_adj[i[1]][i[0]] = 1

    adj = aug_adj.to_sparse()
    return adj


def preprocess_features(features):
    """Row-normalize feature matrix and convert to tuple representation"""
    rowsum = np.array(features.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    features = r_mat_inv.dot(features)
    return features.todense(), sparse_to_tuple(features)


def normalize_adj_tensor(raw_adj, dataset='BlogCatalog'):
    # if dataset =='BlogCatalog':
    #     return raw_adj
    adj = raw_adj[0, :, :]
    row_sum = torch.sum(adj, 0)
    r_inv = torch.pow(row_sum, -0.5).flatten()
    r_inv[torch.isinf(r_inv)] = 0.
    adj = torch.mm(adj, torch.diag_embed(r_inv))
    adj = torch.mm(torch.diag_embed(r_inv), adj)
    adj = adj.unsqueeze(0)
    return adj


def normalize_score(ano_score):
    ano_score = ((ano_score - np.min(ano_score)) / (
            np.max(ano_score) - np.min(ano_score)))
    return ano_score


def process_dis(init_value, cutting_dis_array):
    r_inv = np.power(init_value, -0.5).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    cutting_dis_array = cutting_dis_array.dot(sp.diags(r_inv))
    cutting_dis_array = sp.diags(r_inv).dot(cutting_dis_array)
    return cutting_dis_array


def normalize_adj(adj):
    """Symmetrically normalize adjacency matrix."""
    adj = sp.coo_matrix(adj)
    rowsum = np.array(adj.sum(1))
    d_inv_sqrt = np.power(rowsum, -0.5).flatten()
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    d_mat_inv_sqrt = sp.diags(d_inv_sqrt)
    return adj.dot(d_mat_inv_sqrt).transpose().dot(d_mat_inv_sqrt).tocoo()


def dense_to_one_hot(labels_dense, num_classes):
    """Convert class labels from scalars to one-hot vectors."""
    num_labels = labels_dense.shape[0]
    index_offset = np.arange(num_labels) * num_classes
    labels_one_hot = np.zeros((num_labels, num_classes))
    labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
    return labels_one_hot


def load_mat(dataset):
    """Load .mat dataset."""

    #file_path = f"/content/drive/MyDrive/pes_sem4_basepaper1/data/{dataset}.mat"
    file_path = Path.cwd() / "data" / f"{dataset}.mat"

    
    #data = sio.loadmat("./data/{}.mat".format(dataset))
    data = sio.loadmat(file_path)
    label = data['Label'] if ('Label' in data) else data['gnd']
    attr = data['Attributes'] if ('Attributes' in data) else data['X']
    network = data['Network'] if ('Network' in data) else data['A']
    adj = sp.csr_matrix(network)
    feat = sp.lil_matrix(attr)

    ano_labels = np.squeeze(np.array(label))
    adj1 = adj
    adj2 = None
    return adj1, adj2, feat, ano_labels


from scipy import sparse
def load_dblp_graph():
    if not os.path.exists('./data/DBLP_anomaly.mat'):
        data = sio.loadmat('./data/DBLP')
        features = data['feature'].astype(np.float64)
        label = np.zeros((features.shape[0], 1))
        p = 0.07
        index = np.random.choice(features.shape[0], int(p * features.shape[0]), replace=False)
        label[index] = 1
        feature_anomaly_idx = index[:index.shape[0]//2]
        edge_anomaly_idx = index[index.shape[0]//2:]
        features[feature_anomaly_idx] += np.random.normal(2, 10, size=(feature_anomaly_idx.shape[0], features.shape[1]))
        adj1 = data['net_APA'] + np.random.binomial(1, 0.5, size=(edge_anomaly_idx.shape[0], data['net_APA'].shape[1]))
        adj2 = data['net_APCPA'] + np.random.binomial(1, 0.5, size=(edge_anomaly_idx.shape[0], data['net_APCPA'].shape[1]))
        data = {'feature': features, 'label': label, 'adj1': adj1, 'adj2': adj2}
        sio.savemat('./data/dblp_anomaly.mat', data)
    else:
        data = sio.loadmat('./data/dblp_anomaly.mat')
        features = data['feature']
        adj1 = data['adj1']
        adj2 = data['adj2']
        label = data['label'].reshape(-1, )
    # print('size of anomalies:', sum(label))
    # features = sparse.csr_matrix(features)
    adj1 = sparse.csr_matrix(adj1)
    adj2 = sparse.csr_matrix(adj2)
    return adj1, adj2, features, label


def load_imdb_graph():
    if not os.path.exists('./data/imdb5k_anomaly.mat'):
        data = sio.loadmat('./data/imdb5k.mat')
        features = data['feature'].astype(np.float64)
        label = np.zeros((features.shape[0], 1))
        p = 0.07
        index = np.random.choice(features.shape[0], int(p * features.shape[0]), replace=False)
        label[index] = 1
        feature_anomaly_idx = index[:index.shape[0]//2]
        edge_anomaly_idx = index[index.shape[0]//2:]
        features[feature_anomaly_idx] += np.random.normal(2, 10, size=(feature_anomaly_idx.shape[0], features.shape[1]))
        adj1 = data['MAM'] + np.random.binomial(1, 0.5, size=(edge_anomaly_idx.shape[0], data['MAM'].shape[1]))
        adj2 = data['MDM'] + np.random.binomial(1, 0.5, size=(edge_anomaly_idx.shape[0], data['MDM'].shape[1]))
        data = {'feature': features, 'label': label, 'adj1': adj1, 'adj2': adj2}
        sio.savemat('./data/imdb5k_anomaly.mat', data)
    else:
        data = sio.loadmat('./data/imdb5k_anomaly.mat')
        features = data['feature']
        adj1 = data['adj1']
        adj2 = data['adj2']
        label = data['label'].reshape(-1, )
    # print('size of anomalies:', sum(label))
    # features = sparse.csr_matrix(features)
    adj1 = sparse.csr_matrix(adj1)
    adj2 = sparse.csr_matrix(adj2)
    return adj1, adj2, features, label


def load_cert_graph(d=100):
    v2 = pkl.load(open('./CERT/logon.pkl', 'rb'))
    v1 = pkl.load(open('./CERT/email.pkl', 'rb'))
    malicious_user = pkl.load(open('./CERT/label.pkl', 'rb'))['label']
    label = []
    email_pc_dict = v2['pc_dict']
    email_user_dict = v2['user_dict']
    overlapped_idx = []
    for item, key in email_pc_dict.items():
        if item in v1['pc_dict']:
            overlapped_idx.append(v1['pc_dict'][item])
    v2['graph'] = v2['graph'][overlapped_idx, :]
    v2['weight'] = v2['weight'][overlapped_idx, :]
    overlapped_idx = []
    for item, key in email_user_dict.items():
        if item in v1['user_dict']:
            overlapped_idx.append(v1['user_dict'][item])
        if item in malicious_user:
            label.append(1)
        else:
            label.append(0)
    v2['graph'] = v2['graph'][:, overlapped_idx]
    v2['weight'] = v2['weight'][:, overlapped_idx]
    if not os.path.exists('./CERT/logon_edge_list.txt'):
        n_nodes = v2['weight'].shape[0]
        with open('./CERT/logon_edge_list.txt', 'w') as f:
            for i in range(len(v2['graph'])):
                idx = np.nonzero(v2['graph'][i, :])[0]
                for j in idx:
                    f.write('{} {}\n'.format(i, j + n_nodes))
    if not os.path.exists('./CERT/email_edge_list.txt'):
        n_nodes = v1['weight'].shape[0]
        with open('./CERT/email_edge_list.txt', 'w') as f:
            for i in range(len(v1['graph'])):
                idx = np.nonzero(v1['graph'][i, :])[0]
                for j in idx:
                    f.write('{} {}\n'.format(i, j + n_nodes))
    if not os.path.exists('./CERT/email_edge_list_emb_{}'.format(d)):
        os.system("python ./deepwalk/main.py --representation-size {} --input ./CERT/email_edge_list.txt"
                  " --output ./CERT/email_edge_list_emb_{}".format(d, d))
    if not os.path.exists('./CERT/logon_edge_list_emb_{}'.format(d)):
        os.system("python ./deepwalk/main.py --representation-size {} --input ./CERT/logon_edge_list.txt"
                  " --output ./CERT/logon_edge_list_emb_{}".format(d, d))
    v1_feature = np.zeros((v1['weight'].shape[0], d))
    v2_feature = np.zeros((v2['weight'].shape[0], d))
    with open('./CERT/logon_edge_list_emb_{}'.format(d), 'r') as f:
        next(f)
        for line in f.readlines():
            line = list(map(float, line.split()))
            if line[0] >= 1000:
                v1_feature[int(line[0]) - 1000] = line[1:]
    with open('./CERT/email_edge_list_emb_{}'.format(d), 'r') as f:
        next(f)
        for line in f.readlines():
            line = list(map(float, line.split()))
            if line[0] >= 1000:
                v2_feature[int(line[0]) - 1000] = line[1:]
    v1_feats = v1_feature
    v2_feats = v2_feature
    # v1_adj = np.dot(v1['graph'], v1['graph'].transpose())
    # v2_adj = np.dot(v2['graph'], v2['graph'].transpose())
    v1_adj = v1['graph']
    v2_adj = v2['graph']
    label = np.array(label).reshape(-1, 1)
    adj1 = sparse.csr_matrix(v1_adj)
    adj2 = sparse.csr_matrix(v2_adj)
    return adj1, adj2, v2_feats, label

'''
# compute the distance between each node
def calc_distance(adj, seq):
    dis_array = torch.zeros((adj.shape[0], adj.shape[1]))
    row = adj.shape[0]
    for i in range(row):
        # print(i)
        node_index = torch.argwhere(adj[i, :] > 0)
        for j in node_index:
            dis = torch.sqrt(torch.sum((seq[i] - seq[j]) * (seq[i] - seq[j])))
            dis_array[i][j] = dis
    return dis_array
'''


def calc_distance(adj, seq):
    # adj: (N,N) torch tensor; seq: (N,D)
    idx = (adj > 0).nonzero(as_tuple=False)  # updated here
    if idx.numel() == 0:  # updated here
        return torch.zeros_like(adj, device=seq.device)  # updated here
    src = idx[:, 0]  # updated here
    dst = idx[:, 1]  # updated here
    diff = seq[src] - seq[dst]  # updated here
    dis = torch.sqrt((diff * diff).sum(1))  # updated here
    dis_array = torch.zeros_like(adj, device=seq.device)  # updated here
    dis_array[src, dst] = dis  # updated here
    return dis_array  # updated here


def get_cos_similar(v1: list, v2: list):
    num = float(np.dot(v1, v2))  # 向量点乘
    denom = np.linalg.norm(v1) * np.linalg.norm(v2)  # 求模长的乘积
    return 0.5 + 0.5 * (num / denom) if denom != 0 else 0


def calc_sim(adj_matrix, attr_matrix):
    row = adj_matrix.shape[0]
    col = adj_matrix.shape[1]
    dis_array = np.zeros((row, col))
    for i in range(row):
        # print(i)
        node_index = np.argwhere(adj_matrix[i, :] > 0)[:, 0]
        for j in node_index:
            dis = get_cos_similar(attr_matrix[i].tolist(), attr_matrix[j].tolist())
            dis_array[i][j] = dis

    return dis_array

"""
def graph_nsgt(dis_array, adj):
    #dis_array = dis_array.cuda()
    dis_array = dis_array.to(adj.device)  # updated here (was: dis_array = dis_array.cuda())
    row = dis_array.shape[0]
    dis_array_u = dis_array * adj
    mean_dis = dis_array_u[dis_array_u != 0].mean()
    for i in range(row):
        node_index = torch.argwhere(adj[i, :] > 0)
        if node_index.shape[0] != 0:
            max_dis = dis_array[i, node_index].max()
            min_dis = mean_dis
            if max_dis > min_dis:
                random_value = (max_dis - min_dis) * np.random.random_sample() + min_dis
                cutting_edge = torch.argwhere(dis_array[i, node_index[:, 0]] > random_value)
                if cutting_edge.shape[0] != 0:
                    adj[i, node_index[cutting_edge[:, 0]]] = 0
    adj = adj + adj.T
    adj[adj > 1] = 1
    return adj
"""
def graph_nsgt(dis_array, adj):
    dis_array = dis_array.to(adj.device)  # updated here (was: dis_array = dis_array.cuda())
    row = dis_array.shape[0]
    dis_array_u = dis_array * adj
    nonzero = dis_array_u[dis_array_u != 0]  # updated here (guard for empty case)
    mean_dis = nonzero.mean() if nonzero.numel() > 0 else torch.tensor(0.0, device=adj.device)  # updated here
    for i in range(row):
        node_index = torch.argwhere(adj[i, :] > 0)
        if node_index.shape[0] != 0:
            max_dis = dis_array[i, node_index].max()
            min_dis = mean_dis
            if max_dis > min_dis:
                random_value = (max_dis - min_dis) * np.random.random_sample() + float(min_dis.item())  # updated here
                cutting_edge = torch.argwhere(dis_array[i, node_index[:, 0]] > random_value)
                if cutting_edge.shape[0] != 0:
                    adj[i, node_index[cutting_edge[:, 0]]] = 0
    adj = adj + adj.T
    adj[adj > 1] = 1
    return adj


import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
plt.rcParams['figure.dpi'] = 300
plt.rcParams['figure.figsize'] = (8.5, 7.5)
from matplotlib.backends.backend_pdf import PdfPages

def draw_pdf(message, ano_label, dataset):
    with PdfPages('{}-TAM.pdf'.format(dataset)) as pdf:
        normal_message_all = message[ano_label == 0]
        abnormal_message_all = message[ano_label == 1]
        message_all = [normal_message_all, abnormal_message_all]
        mu_0 = np.mean(message_all[0])
        sigma_0 = np.std(message_all[0])
        print('The mean of normal {}'.format(mu_0))
        print('The std of normal {}'.format(sigma_0))
        mu_1 = np.mean(message_all[1])
        sigma_1 = np.std(message_all[1])
        print('The mean of abnormal {}'.format(mu_1))
        print('The std of abnormal {}'.format(sigma_1))
        n, bins, patches = plt.hist(message_all, bins=30, normed=1, label=['Normal', 'Abnormal'])
        y_0 = mlab.normpdf(bins, mu_0, sigma_0)
        y_1 = mlab.normpdf(bins, mu_1, sigma_1)
        plt.plot(bins, y_0, color='steelblue', linestyle='--', linewidth=7.5)
        plt.plot(bins, y_1, color='darkorange', linestyle='--', linewidth=7.5)
        plt.yticks(fontsize=30)
        plt.xticks(fontsize=30)
        plt.legend(loc='upper left', fontsize=30)
        plt.title(''.format(dataset), fontsize=25)
        plt.show()


def draw_pdf_str_attr(message, ano_label, str_ano_label, attr_ano_label, dataset):
    with PdfPages('{}-TAM.pdf'.format(dataset)) as pdf:
        normal_message_all = message[ano_label == 0]
        str_abnormal_message_all = message[str_ano_label == 1]
        attr_abnormal_message_all = message[attr_ano_label == 1]
        message_all = [normal_message_all, str_abnormal_message_all, attr_abnormal_message_all]

        mu_0 = np.mean(message_all[0])
        sigma_0 = np.std(message_all[0])
        print('The mean of normal {}'.format(mu_0))
        print('The std of normal {}'.format(sigma_0))
        mu_1 = np.mean(message_all[1])
        sigma_1 = np.std(message_all[1])
        print('The mean of str_abnormal {}'.format(mu_1))
        print('The std of str_abnormal {}'.format(sigma_1))
        mu_2 = np.mean(message_all[2])
        sigma_2 = np.std(message_all[2])
        print('The mean of attt_abnormal {}'.format(mu_2))
        print('The std of attt_abnormal {}'.format(sigma_2))
        n, bins, patches = plt.hist(message_all, bins=30, normed=1, label=['Normal', 'Structural Abnormal', 'Contextual Abnormal'])
        y_0 = mlab.normpdf(bins, mu_0, sigma_0)
        y_1 = mlab.normpdf(bins, mu_1, sigma_1)
        y_2= mlab.normpdf(bins, mu_2, sigma_2)  #

        plt.plot(bins, y_0, color='steelblue', linestyle='--', linewidth=3.5)
        plt.plot(bins, y_1, color='darkorange', linestyle='--', linewidth=3.5)
        plt.plot(bins, y_2, color='green', linestyle='--', linewidth=3.5)

        plt.xlabel('TAM-based Affinity', fontsize=25)
        plt.ylabel('Number of Samples', size=25)
        plt.yticks(fontsize=25)
        plt.xticks(fontsize=25)
        plt.legend(loc='upper left', fontsize=18)
        plt.title('{}'.format(dataset), fontsize=25)
        plt.show()
print('run of this cell completed');

run of this cell completed


In [17]:
################################################################## model.py
import torch.nn as nn
import torch.nn.functional as F
#from utils import *
from sklearn.metrics import average_precision_score, roc_auc_score


class DiffPoolingBlock(torch.nn.Module):
    def __init__(self, in_dim, n_clusters, tau=2, dim=-1, n_layer=1):
        super().__init__()
        self.linear = nn.ModuleList()
        self.linear.append(nn.Linear(in_dim, n_clusters))
        for i in range(n_layer-1):
            self.linear.append(nn.Linear(n_clusters, n_clusters))
        self.softmax = nn.Softmax(dim=dim)
        self.activation = nn.ReLU()
        self.tau = tau
        self.dim = dim

    def reset_parameters(self):
        for layer in self.linear:
            layer.reset_parameters()

    def forward(self, h, adj):
        out = h
        for i in range(len(self.linear)):
            out = self.activation(torch.mm(adj, self.linear[i](out)) + 1e-15)
        return self.softmax(out/self.tau)


class GCN(nn.Module):
    def __init__(self, in_ft, out_ft, act, bias=True):
        super(GCN, self).__init__()
        self.fc = nn.Linear(in_ft, out_ft, bias=False)
        self.act = nn.PReLU() if act == 'prelu' else act

        if bias:
            self.bias = nn.Parameter(torch.FloatTensor(out_ft))
            self.bias.data.fill_(0.0)
        else:
            self.register_parameter('bias', None)

        for m in self.modules():
            self.weights_init(m)

    def weights_init(self, m):
        if isinstance(m, nn.Linear):
            torch.nn.init.xavier_uniform_(m.weight.data)
            if m.bias is not None:
                m.bias.data.fill_(0.0)

    def forward(self, seq, adj, sparse=False):
        seq_fts = self.fc(seq)
        if sparse:
            out = torch.unsqueeze(torch.spmm(adj, torch.squeeze(seq, 0)), 0)
        else:
            out = torch.mm(adj, seq_fts)
        if self.bias is not None:
            out += self.bias

        return self.act(out)


class AvgReadout(nn.Module):
    def __init__(self):
        super(AvgReadout, self).__init__()

    def forward(self, seq):
        return torch.mean(seq, 1)


class MaxReadout(nn.Module):
    def __init__(self):
        super(MaxReadout, self).__init__()

    def forward(self, seq):
        return torch.max(seq, 1).values


class MinReadout(nn.Module):
    def __init__(self):
        super(MinReadout, self).__init__()

    def forward(self, seq):
        return torch.min(seq, 1).values


class WSReadout(nn.Module):
    def __init__(self):
        super(WSReadout, self).__init__()

    def forward(self, seq, query):
        query = query.permute(0, 2, 1)
        sim = torch.matmul(seq, query)
        sim = F.softmax(sim, dim=1)
        sim = sim.repeat(1, 1, 64)
        out = torch.mul(seq, sim)
        out = torch.sum(out, 1)
        return out


class Model(nn.Module):
    def __init__(self, n_in, n_h, activation, readout, config):
        super(Model, self).__init__()
        self.read_mode = readout
        self.gcn1 = GCN(n_in, 2 * n_h, activation)
        self.gcn2 = GCN(2 * n_h, n_h, activation)
        self.act = nn.PReLU()
        self.fc1 = nn.Linear(n_h, 2 * n_h, bias=False)
        self.diffpool = DiffPoolingBlock(n_in, n_clusters=config['clusters'], dim=-1)
        self.ReLU = nn.ReLU()
        if readout == 'max':
            self.read = MaxReadout()
        elif readout == 'min':
            self.read = MinReadout()
        elif readout == 'avg':
            self.read = AvgReadout()
        elif readout == 'weighted_sum':
            self.read = WSReadout()
        self.tau = 1
        self.lamb = config['lamb']
        self.alpha = config['alpha']
        self.beta = config['beta']
        self.norm = config['norm']

    def normalize_adj_tensor(self, adj):
        row_sum = torch.sum(adj, 0)
        r_inv = torch.pow(row_sum, -0.5).flatten()
        r_inv = torch.nan_to_num(r_inv, posinf=0, neginf=0)
        adj = torch.mm(adj, torch.diag_embed(r_inv))
        adj = torch.mm(torch.diag_embed(r_inv), adj)
        return adj

    def forward(self, seq, adj, raw_adj, adj2=None,  raw_adj2=None, sparse=False):
        if adj2 is not None:
            feat1 = self.gcn2(self.gcn1(seq, adj[0]), adj[0])
            feat2 = self.gcn2(self.gcn1(seq, adj2[0]), adj2[0])
            feat = (feat1 + feat2)/2
            s_adj = (adj[0] + adj2[0])/2
        else:
            feat = self.gcn2(self.gcn1(seq, adj[0]), adj[0])
            s_adj = adj[0]
            raw_adj2 = raw_adj
        assign = self.diffpool(seq, s_adj)
        self.G = assign
        raw_adj = (raw_adj[0] + raw_adj2[0])/2
        con_loss = self.graph_contrastive_loss(feat, feat, assign, raw_adj)
        cluster_sim = self.alpha * torch.mm(assign, assign.T) + raw_adj * (1 - self.alpha)
        if self.norm:
            cluster_sim = self.normalize_adj_tensor(cluster_sim)
        if adj2 is None:
            view_consistency = 0
        else:
            view_consistency = torch.norm(feat1 - feat2, dim=1, p=2).mean()
        loss, message_sum1 = self.max_message(feat, cluster_sim)
        fc1 = self.fc1(feat)
        if self.beta != 0:
            loss += self.beta * self.reg_edge(fc1, adj[0])
        return feat, cluster_sim, loss + self.lamb * (con_loss + view_consistency)

    def graph_contrastive_loss(self, v1, v2, assignment, adj, test_phase=False):
        if v1.shape[0] > 10000 and not test_phase:
            idx = torch.randperm(v1.shape[0])[:8000]
        else:
            idx = torch.arange(v1.shape[0])
        v1, v2, assignment = v1[idx], v2[idx], assignment[idx]
        adj = adj[idx, :][:, idx]
        v = (v1 + v2)/2
        sim = self.cosine_sim(v, v, self.tau)
        O = self.alpha * torch.matmul(assignment, assignment.T) + (1 - self.alpha) * adj
        n = assignment.shape[0]
        loss = torch.norm(O / O.sum(dim=1).clamp_min(0.01) - sim, 2, dim=1).mean()
        return loss

    def view_consistency(self, seq, adj, adj2, sparse=False):
        if adj2 is None:
            return 0
        feat1 = self.gcn2(self.gcn1(seq, adj[0]), adj[0])
        feat2 = self.gcn2(self.gcn1(seq, adj2[0]), adj2[0])
        view_consistency = torch.norm(feat1 - feat2, dim=1, p=2)
        return view_consistency * self.lamb

    def cosine_sim(self, x1, x2, eps=1e-15, temperature=1):
        w1 = x1.norm(p=2, dim=1, keepdim=True)
        w2 = x2.norm(p=2, dim=1, keepdim=True)
        return torch.matmul(x1, x2.t()) / ((w1 * w2.t()).clamp(min=eps) * temperature)

    def reg_edge(self, emb, adj):
        emb = emb / torch.norm(emb, dim=-1, keepdim=True)
        sim_u_u = torch.mm(emb, emb.T)
        adj_inverse = (1 - adj)
        sim_u_u = sim_u_u * adj_inverse
        sim_u_u_no_diag = torch.sum(sim_u_u, 1)
        row_sum = torch.sum(adj_inverse, 1)
        r_inv = torch.pow(row_sum, -1)
        r_inv[torch.isinf(r_inv)] = 0.
        sim_u_u_no_diag = sim_u_u_no_diag * r_inv
        loss_reg = torch.sum(sim_u_u_no_diag)
        return loss_reg

    def max_message(self, feature, adj_matrix):
        feature = feature / torch.norm(feature, dim=-1, keepdim=True)
        sim_matrix = torch.mm(feature, feature.T)
        sim_matrix = torch.squeeze(sim_matrix) * adj_matrix
        sim_matrix[torch.isinf(sim_matrix)] = 0
        sim_matrix[torch.isnan(sim_matrix)] = 0
        row_sum = torch.sum(adj_matrix, 0)
        r_inv = torch.pow(row_sum, -1).flatten()
        r_inv[torch.isinf(r_inv)] = 0.
        message = torch.sum(sim_matrix, 1)
        message = message * r_inv
        return - torch.sum(message), message

    def inference(self, feature, adj_matrix):
        feature = feature / torch.norm(feature, dim=-1, keepdim=True)
        sim_matrix = torch.mm(feature, feature.T)
        sim_matrix = torch.squeeze(sim_matrix) * adj_matrix
        row_sum = torch.sum(adj_matrix, 0)
        r_inv = torch.pow(row_sum, -1).flatten()
        r_inv[torch.isinf(r_inv)] = 0.
        message = torch.sum(sim_matrix, 1)
        message = message * r_inv
        return message

    def evaluation(self, score, ano_label):
        auc = roc_auc_score(ano_label, score)
        AP = average_precision_score(ano_label, score, average='macro', pos_label=1, sample_weight=None)
        print('AP::{:.4f}, AUC:{:.4f}'.format(AP, auc))
print('run of this cell completed');

run of this cell completed


In [20]:
######################################################################################################################################################################################################################VERSION @
print('run of this cell started');
# ============================== main.py (CPU-only) ==============================
import os
import os.path
import argparse
import time

import numpy as np
import torch

#from utils import *          # provides: load_dataset, graph_nsgt, normalize_adj_tensor, calc_distance, normalize_score
#from model import Model      # model class

# ----- Force CPU -----
#os.environ["CUDA_VISIBLE_DEVICES"] = ""

device = device #torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("Using device:", device)


# Where to cache distances / outputs
save_path = save_path


run of this cell started
Using device: cpu


In [22]:


def main(args):
    # Repro
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)

    # Load and preprocess data
    raw_features, features, adj1, adj2, ano_label, raw_adj1, raw_adj2, config = load_dataset(args)

    # Build models & optimizers (use config['cutting'] everywhere)
    optimiser_list, model_list = [], []
    for _ in range(config['cutting']):
        model = Model(config['ft_size'], args.embedding_dim, 'prelu', args.readout, config).to(device)
        optimiser = torch.optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)
        optimiser_list.append(optimiser)
        model_list.append(model)

    start = time.time()
    total_epoch = args.num_epoch * config['cutting']

    # Initial cut-adj (view 1)
    new_adj_list1 = [raw_adj1]
    all_cut_adj1 = torch.cat(new_adj_list1)

    # Distance cache (view 1)
    os.makedirs(save_path, exist_ok=True)
    distance1_file = os.path.join(save_path, f"{args.dataset}_distance1.npy")
    if os.path.exists(distance1_file):
        dis_array1 = torch.tensor(np.load(distance1_file), dtype=torch.float32)
    else:
        dis_array1 = calc_distance(raw_adj1[0, :, :], raw_features[0, :, :])
        np.save(distance1_file, dis_array1.detach().cpu().numpy())

    # Optional second view
    if raw_adj2 is not None:
        new_adj_list2 = [raw_adj2]
        all_cut_adj2 = torch.cat(new_adj_list2)

        dist2_path = os.path.join(save_path, f"{args.dataset}_distance2.npy")
        if os.path.exists(dist2_path):
            dis_array2 = torch.tensor(np.load(dist2_path), dtype=torch.float32)
        else:
            # IMPORTANT: use raw_adj2 for view-2 distance
            dis_array2 = calc_distance(raw_adj2[0, :, :], raw_features[0, :, :])
            np.save(dist2_path, dis_array2.detach().cpu().numpy())
    else:
        all_cut_adj2 = None
        dis_array2 = None

    index = 0
    message_mean_list = []

    # Per-cut training
    for n_cut in range(config['cutting']):
        message_list = []
        optimiser_list[index].zero_grad()
        model_list[index].train()

        # Cut adj for view 1
        cut_adj1 = graph_nsgt(dis_array1, all_cut_adj1[0, :, :])
        cut_adj1 = cut_adj1.unsqueeze(0)
        adj_norm1 = normalize_adj_tensor(cut_adj1, args.dataset)

        # Cut adj for view 2
        if all_cut_adj2 is not None:
            cut_adj2 = graph_nsgt(dis_array2, all_cut_adj2[0, :, :])
            cut_adj2 = cut_adj2.unsqueeze(0)
            adj_norm2 = normalize_adj_tensor(cut_adj2, args.dataset)
        else:
            adj_norm2 = None

        # Epoch loop (clear grads each step)
        for epoch in range(args.num_epoch):
            optimiser_list[index].zero_grad()
            node_emb, cluster_sim, loss = model_list[index].forward(
                features[0], adj_norm1, raw_adj1, adj_norm2, raw_adj2
            )
            loss.backward()
            optimiser_list[index].step()

            if (epoch + 1) % 100 == 0:
                print(
                    f"Epoch [{n_cut * args.num_epoch + epoch + 1}/{total_epoch}], "
                    f"time: {time.time() - start:.4f}, Loss: {loss.item():.4f}"
                )

        # Scoring
        message_sum = (
            model_list[index].inference(node_emb, cluster_sim)
            + model_list[index].view_consistency(features[0], adj_norm1, adj_norm2)
        )
        message_list.append(torch.unsqueeze(message_sum.detach().cpu(), 0))

        # Update bases for next cut
        all_cut_adj1[0, :, :] = torch.squeeze(cut_adj1)
        if all_cut_adj2 is not None:
            all_cut_adj2[0, :, :] = torch.squeeze(cut_adj2)

        index += 1

        # Aggregate & evaluate
        message_list = torch.mean(torch.cat(message_list), 0)
        message_mean_list.append(torch.unsqueeze(message_list, 0))
        message_mean_cut = torch.mean(torch.cat(message_mean_list), 0)
        message_mean = np.array(message_mean_cut.detach())
        score = 1 - normalize_score(message_mean)
        model_list[index - 1].evaluation(score, ano_label)

    print("Total time (s):", time.time() - start)
print('run of this cell completed');

run of this cell completed


In [25]:


if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Cluster-Aware Graph Anomaly Detection (CARE-demo)")
    parser.add_argument('--dataset', type=str, default='Amazon', help='Amazon | BlogCatalog | imdb | dblp')
    parser.add_argument('--lr', type=float, default=1e-5)
    parser.add_argument('--weight_decay', type=float, default=0.0)
    parser.add_argument('--seed', type=int, default=2)
    parser.add_argument('--embedding_dim', type=int, default=128)
    parser.add_argument('--num_epoch', type=int, default=500)
    parser.add_argument('--readout', type=str, default='avg')      # max | min | avg | weighted_sum
    parser.add_argument('--cutting', type=int, default=25)         # kept for CLI parity; config['cutting'] is used internally
    parser.add_argument('--lamb', type=float, default=0.1)
    parser.add_argument('--alpha', type=float, default=0.8)
    parser.add_argument('--clusters', type=int, default=10)

    # Friendly defaults for notebooks / scripts (no CLI args needed)
    args = parser.parse_args(args=[])

    print('Dataset:', args.dataset)
    main(args)
# ===============================================================================



Dataset: Amazon
Epoch [100/12500], time: 197.3624, Loss: -9071.0615
Epoch [200/12500], time: 395.7949, Loss: -9473.6924
Epoch [300/12500], time: 590.5679, Loss: -9718.1484
Epoch [400/12500], time: 786.6324, Loss: -9866.6289
Epoch [500/12500], time: 985.6993, Loss: -9959.4053
AP::0.2151, AUC:0.5731
Epoch [600/12500], time: 1188.4782, Loss: -8502.3740
Epoch [700/12500], time: 1389.1498, Loss: -9178.1084
Epoch [800/12500], time: 1587.3178, Loss: -9573.1719
Epoch [900/12500], time: 1772.4200, Loss: -9795.6006
Epoch [1000/12500], time: 1953.9954, Loss: -9919.8291
AP::0.3033, AUC:0.6359
Epoch [1100/12500], time: 2164.6496, Loss: -8376.7090
Epoch [1200/12500], time: 2363.5132, Loss: -9047.5752
Epoch [1300/12500], time: 2561.8472, Loss: -9472.9902
Epoch [1400/12500], time: 2759.0827, Loss: -9722.7236
Epoch [1500/12500], time: 2943.2562, Loss: -9870.9375
AP::0.3667, AUC:0.6941
Epoch [1600/12500], time: 3131.8009, Loss: -8112.7080
Epoch [1700/12500], time: 3329.5097, Loss: -8837.6719
Epoch [1800

In [ ]:
############
'''
Using device: cpu
Dataset: Amazon
Epoch [100/12500], time: 2040.4267, Loss: -9071.0615
Epoch [200/12500], time: 3990.6654, Loss: -9473.6924
Epoch [300/12500], time: 5898.8861, Loss: -9718.1484
Epoch [400/12500], time: 7811.2736, Loss: -9866.6289
Epoch [500/12500], time: 9725.5304, Loss: -9959.4072
AP::0.2151, AUC:0.5731
Epoch [600/12500], time: 11772.2434, Loss: -8502.3740
Epoch [700/12500], time: 13748.7978, Loss: -9178.1084
Epoch [800/12500], time: 15705.0023, Loss: -9573.1719
Epoch [900/12500], time: 17665.5378, Loss: -9795.6006
Epoch [1000/12500], time: 19604.1226, Loss: -9919.8291
AP::0.3033, AUC:0.6359
Epoch [1100/12500], time: 21619.7950, Loss: -8376.7080
Epoch [1200/12500], time: 23552.9021, Loss: -9047.5762
Epoch [1300/12500], time: 25474.5063, Loss: -9472.9902
Epoch [1400/12500], time: 27392.5535, Loss: -9722.7246
Epoch [1500/12500], time: 29350.0607, Loss: -9870.9375
AP::0.3667, AUC:0.6941
Epoch [1600/12500], time: 31415.8522, Loss: -8112.7080
Epoch [1700/12500], time: 33385.9079, Loss: -8837.6709
Epoch [1800/12500], time: 35374.1808, Loss: -9307.3701
Epoch [1900/12500], time: 37346.4518, Loss: -9596.7598
'''